In [1]:
%pylab inline
import os,sys,pickle
from scipy.stats import percentileofscore, chi2, norm
from scipy.linalg import sqrtm
from scipy.interpolate import interp1d
from scipy.special import loggamma, spherical_jn, sici
from scipy.integrate import simps
from sympy.physics.wigner import wigner_3j
from scipy.optimize import minimize

Populating the interactive namespace from numpy and matplotlib


# Even-Parity BOSS 4PCF Analysis
This notebook contains the main analysis routines used in Philcox++ (2021) to probe gravitational non-Gaussianity in the isotropic connected 4-point correlation function (4PCF) of the BOSS CMASS sample. For questions, email [Oliver Philcox](mailto:ohep2@cantab.ac.uk).

In [2]:
# Data directories (containing pre-computed BOSS and Patchy results)
data_dir = '/home/ophilcox/Parity-Even-4PCF/data/'

# Output plotting directory
plot_dir = '/home/ophilcox/Parity-Even-4PCF/figs/'
if not os.path.exists(plot_dir): os.makedirs(plot_dir)

# Binning parameters
R_min = 20
R_max = 160
n_r = 10

# If True, replace the BOSS data with a single Patchy mock (for blinding reasons)
fake_data = False

# If true, remove any bins separated by <= 14 Mpc/h
apply_cuts = False

# If true, subtract the mean of the signal
remove_mean = False

# 1) Load 4PCFs
- These are computed with the [*encore*](https://github.com/oliverphilcox/encore) code of [Philcox et al. 2021](https://arxiv.org/abs/2105.08722)
- We use 10 linearly-spaced radial bins in the range [20, 160] $h^{-1}\mathrm{Mpc}$
- Multiplets, i.e. choices of $\{\ell_1,\ell_2,\ell_3\}$, are computed up to $\ell_\mathrm{max} = 5$, with $\ell_i=5$ multiplets only being used for edge-correction.
- Here we consider only bins with $\ell_i\leq 4$ to keep the dimensionality reasonable.
- We analyze both the BOSS data, in the CMASS NGC and SGC regions, as well as 1000 MultiDark-Patchy mocks.
- All data is provided in the ```data/``` directory. Note that this additionally includes the isotropic 2PCF and 3PCF measurements from BOSS.

In [3]:
binner = lambda bins: (0.5+bins)*(R_max-R_min)/n_r+R_min

### First Load Patchy data
with np.load(data_dir+'all_patchy_fourpcf.npz') as d:
    # SGC measurements
    fourpcfSall=d['fourpcfSall']
    # NGC measurements
    fourpcfNall=d['fourpcfNall']
    # {r1, r2, r3}
    radii=d['radii']
    # {bin-index 1, bin-index 2, bin-index 3}
    bins=d['bins']
    # {ell-1, ell-2, ell-3}
    ells=d['ells']
    # SGC disconnected measurements
    fourpcfSdisc=d['fourpcfSdisc']
    # NGC disconnected measurements
    fourpcfNdisc=d['fourpcfNdisc']

# Compute the connected-only 4PCF for Patchy
fourpcfN = np.asarray(fourpcfNall) - np.asarray(fourpcfNdisc)
fourpcfS = np.asarray(fourpcfSall) - np.asarray(fourpcfSdisc)

n_radial = len(bins[0])
n_mult = len(ells[0])

### Now load the BOSS data

def load_boss(patch,discon=False,return_all=False):
    """Load precomputed 4PCFs from BOSS, using either the full or disconnected estimators.
    This loads data from the NGC or SGC, here labelled 'N' or 'S'."""
    if discon:
        infile = data_dir+'boss_cmass%s.zeta_discon_4pcf.txt'%patch
    else:
        infile = data_dir+'boss_cmass%s.zeta_4pcf.txt'%patch
    bins1,bins2,bins3 = np.asarray(np.loadtxt(infile,skiprows=3,max_rows=3),dtype=int)
    ell1,ell2,ell3 = np.asarray(np.loadtxt(infile,skiprows=9)[:,:3],dtype=int).T
    fourpcf_boss = np.loadtxt(infile,skiprows=9)[:,3:]
    r1_4pcf = binner(bins1)
    r2_4pcf = binner(bins2)
    r3_4pcf = binner(bins3)  
    if return_all:
        return [r1_4pcf,r2_4pcf,r3_4pcf],[bins1,bins2,bins3],[ell1,ell2,ell3],fourpcf_boss
    else:
        return fourpcf_boss

if fake_data:
    # Replace the BOSS data with a single Patchy mock for blinding
    print("Replacing BOSS data with first Patchy mock")
    fourpcf_bossN = fourpcfN[0]
    fourpcf_bossS = fourpcfS[0]
    fourpcf_bossNdisc = fourpcfNdisc[0]
    fourpcf_bossSdisc = fourpcfSdisc[0]
    fourpcfN = fourpcfN.copy()[1:]
    fourpcfS = fourpcfS.copy()[1:]
else:
    # Load the BOSS data
    radii_boss, bins_boss, ells_boss, fourpcf_bossNall = load_boss('N',return_all=True)
    fourpcf_bossSall = load_boss('S')
    fourpcf_bossNdisc = load_boss('N',discon=True)
    fourpcf_bossSdisc = load_boss('S',discon=True)
    
    # Remove any odd multipoles!
    odds = np.asarray([(-1)**(ells_boss[0][i]+ells_boss[1][i]+ells_boss[2][i])==-1 for i in range(len(fourpcf_bossNall))])
    fourpcf_bossNall = np.asarray(fourpcf_bossNall)[~odds]
    fourpcf_bossSall = np.asarray(fourpcf_bossSall)[~odds]

    fourpcf_bossN = np.asarray(fourpcf_bossNall) - np.asarray(fourpcf_bossNdisc)
    fourpcf_bossS = np.asarray(fourpcf_bossSall) - np.asarray(fourpcf_bossSdisc)

if remove_mean:
    # Remove the mean signal for testing
    print("Removing mean of data")
    meanN = np.mean(fourpcfN,axis=0)
    meanS = np.mean(fourpcfS,axis=0)
    for i in range(len(fourpcfN)):
        fourpcfN[i] = fourpcfN[i]+meanN*-1
        fourpcfS[i] = fourpcfS[i]+meanS*-1
    fourpcf_bossN = fourpcf_bossN+meanN*-1
    fourpcf_bossS = fourpcf_bossS+meanS*-1
    
n_mocks = len(fourpcfN)
assert n_mocks == len(fourpcfS)

print("N_mocks: %d"%n_mocks)
print("N_Lambda: %d"%n_mult)
print("N_radial: %d"%n_radial)

N_mocks: 1000
N_Lambda: 69
N_radial: 120


# 2) Apply Data Cuts

- Apply the cuts described above for the angular bins.
- We can optionally also filter out any bins with internal separations less than 14 Mpc/h, which are diffiult to model.

In [4]:
# filter out ells which are not properly edge-corrected
LMAX=4
ang_filt = np.asarray(np.logical_and(np.logical_and(ells[0]<=LMAX,ells[1]<=LMAX),ells[2]<=LMAX))

# Optionally remove any bins within 14 Mpc of each other
dr = (R_max-R_min)/n_r
radial_filt = ((radii[1]-radii[0])>1.9*n_r)&((radii[2]-radii[1])>1.9*n_r)
if apply_cuts:
    print("Applying radial binning cuts")
else:
    radial_filt = np.ones_like(radial_filt)

# Apply filters
filt_fourpcfN = np.asarray([ff[ang_filt][:,radial_filt] for ff in fourpcfN])
filt_fourpcfS = np.asarray([ff[ang_filt][:,radial_filt] for ff in fourpcfS])
filt_fourpcf_bossN = fourpcf_bossN[ang_filt][:,radial_filt]
filt_fourpcf_bossS = fourpcf_bossS[ang_filt][:,radial_filt]
filt_flat_fourpcfN = np.asarray([ff.ravel() for ff in filt_fourpcfN])
filt_flat_fourpcfS = np.asarray([ff.ravel() for ff in filt_fourpcfS])
filt_flat_fourpcf_bossN = filt_fourpcf_bossN.ravel()
filt_flat_fourpcf_bossS = filt_fourpcf_bossS.ravel()

# Repeat for disconnected piece
filt_fourpcfNdisc = np.asarray([ff[ang_filt][:,radial_filt] for ff in fourpcfNdisc])
filt_fourpcfSdisc = np.asarray([ff[ang_filt][:,radial_filt] for ff in fourpcfSdisc])
filt_fourpcf_bossNdisc = fourpcf_bossNdisc[ang_filt][:,radial_filt]
filt_fourpcf_bossSdisc = fourpcf_bossSdisc[ang_filt][:,radial_filt]
filt_flat_fourpcfNdisc = np.asarray([ff.ravel() for ff in filt_fourpcfNdisc])
filt_flat_fourpcfSdisc = np.asarray([ff.ravel() for ff in filt_fourpcfSdisc])
filt_flat_fourpcf_bossNdisc = filt_fourpcf_bossNdisc.ravel()
filt_flat_fourpcf_bossSdisc = filt_fourpcf_bossSdisc.ravel()

# Redefine array dimensions
n_radial = np.sum(radial_filt)
n_mult = np.sum(ang_filt)

print("N_Lambda: %d"%n_mult)
print("N_bins: %d"%n_radial)
print("N_total: %d"%(n_mult*n_radial))

N_Lambda: 42
N_bins: 120
N_total: 5040


# 3) Plot Correlation and Covariance Matrices
- Theory covariances are computed in the Gaussian Random Field approximation as in Hou et al. (in prep.).
- Note that they do not include redshift-space distortions, non-Gaussian contributions, or a proper treatment of the survey geometry.
- For this reason they are used only as approximate tools for the compressed-Gaussian analyses below; importantly, we do *not* assume them to be equal to the true covariance of the data.

In [ ]:
### Compute Patchy covariances
print("Computing Patchy covariances")
corrN = np.corrcoef(filt_flat_fourpcfN.T)
covN = np.cov(filt_flat_fourpcfN.T)
corrS = np.corrcoef(filt_flat_fourpcfS.T)
covS = np.cov(filt_flat_fourpcfS.T)

### Load theory covariances
def load_theory_cov(patch):
    cov_file = data_dir+'gaussian_cov_patchy_%s.cov'%patch
    cov_dat = pickle.load(open(cov_file, "rb"))

    # Construct covariance
    theory_cov = np.zeros((n_mult*n_radial,n_mult*n_radial))
    for i in range(n_mult):
        lam1 = '%d%d%d'%(ells[0][ang_filt][i],ells[1][ang_filt][i],ells[2][ang_filt][i])
        for j in range(n_mult):
            lam2 = '%d%d%d'%(ells[0][ang_filt][j],ells[1][ang_filt][j],ells[2][ang_filt][j])
            try: 
                this_cov = cov_dat[lam1,lam2]
            except KeyError:
                try:
                    this_cov = cov_dat[lam2,lam1].T
                except KeyError:
                    print("Covariance element %s,%s has not been computed!"%(lam1,lam2))
            theory_cov[i*n_radial:(i+1)*n_radial,j*n_radial:(j+1)*n_radial] = this_cov[radial_filt][:,radial_filt]
    return theory_cov

print("Loading theory covariances")
theory_covN = load_theory_cov('ngc')
theory_covS = load_theory_cov('sgc')

# Compute the correlation matrices 
theory_corrN = theory_covN/np.sqrt(np.outer(np.diag(theory_covN),np.diag(theory_covN)))
theory_corrS = theory_covS/np.sqrt(np.outer(np.diag(theory_covS),np.diag(theory_covS)))

# Invert the theory covariance
print("Computing inverse theory covariances")
inv_theory_covN = np.linalg.inv(theory_covN)
inv_theory_covS = np.linalg.inv(theory_covS)

# Define the eigenvectors from the inverse theory covariance
print("Performing eigendecomposition")
evalsN,evecsN = np.linalg.eigh(inv_theory_covN)
evalsS,evecsS = np.linalg.eigh(inv_theory_covS)

Computing Patchy covariances
Loading theory covariances
Computing inverse theory covariances


**Plot Correlation Matrices**
- Correlation matrices are defined as $R_{ij} = C_{ij}/\sqrt{C_{ii}C_{jj}}$
- Each submatrix (indicated by the dotted lines) gives a different multiplet, as indicated in green. For example, the second-to-left submatrix on the top line is the correlation of $\zeta_{000}$ and $\zeta_{011}$. 

In [ ]:
# Maximum number of multiplets to plot
n_max = 8

fig,ax = plt.subplots(1,2,figsize=(12,6))
# Sample correlation
im = ax[0].imshow(corrN[:n_max*n_radial,:n_max*n_radial],vmax=1,vmin=-1,cmap=cm.RdBu_r);
# Theory correlation
im = ax[1].imshow(theory_corrN[:n_max*n_radial,:n_max*n_radial],vmax=1,vmin=-1,cmap=cm.RdBu_r);
for i in range(n_max):
    for a in range(2):
        if i!=0: ax[a].hlines(n_radial*i-1,0,n_max*n_radial,linestyles='--',alpha=0.4)
        if i!=0: ax[a].vlines(n_radial*i-1,0,n_max*n_radial,linestyles='--',alpha=0.4)
        if i!=n_max-1: ax[a].text(n_radial*(n_max-1)+n_radial/10+20,n_radial*i+n_radial/2,'%d%d%d'%(ells[0][ang_filt][i],ells[1][ang_filt][i],ells[2][ang_filt][i]),color='darkgreen',size=10)
        ax[a].text(n_radial*i+n_radial/10,n_radial*(n_max-1)+n_radial/2+20,'%d%d%d'%(ells[0][ang_filt][i],ells[1][ang_filt][i],ells[2][ang_filt][i]),color='darkgreen',size=10)
for a in range(2):
    ax[a].set_xlim([0,n_radial*n_max-1])
    ax[a].set_ylim([n_radial*n_max-1,0])
ax[0].set_title('%d Patchy Mocks'%n_mocks,fontsize=14)
ax[1].set_title('Gaussian Theory',fontsize=14);
for a in range(2): ax[a].set_xlabel(r'Bin Index 1',fontsize=14)
ax[0].set_ylabel(r'Bin Index 2',fontsize=14)

fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.95, 0.15, 0.03, 0.7])
cbar=fig.colorbar(im, cax=cbar_ax)
cbar.set_label('Correlation Matrix',fontsize=14)

fig.savefig(plot_dir+'correlation_comparison_ngc.pdf',bbox_inches='tight')

**Plot Variances**
- We also plot the diagonal elements of the matrices, *i.e.* the 4PCF variances.

In [ ]:
plt.figure(figsize=(8,4))
# Sample variance
plt.plot(np.diag(covN)[:n_max*n_radial],c='r',ls='-',label='%d Patchy Mocks'%n_mocks)
# Theory variance
plt.plot(np.diag(theory_covN)[:n_max*n_radial],c='b',ls='--',label='Gaussian Theory')
plt.yscale('log')
plt.xlim([0,n_max*n_radial])
ylims = [1e-8,1e-4]
plt.ylim(ylims)
for i in range(n_max):
    if i!=0: plt.vlines(n_radial*i,ylims[0],ylims[1],linestyles='--',alpha=0.4)
    plt.text(n_radial*i+n_radial/10,ylims[0]*1.3,'%d%d%d'%(ells[0][ang_filt][i],ells[1][ang_filt][i],ells[2][ang_filt][i]),color='darkgreen',size=10) 
plt.legend(loc='upper left',fontsize=12)
plt.xlabel(r'Bin Index',fontsize=13)
plt.ylabel(r'Covariance Diagonal',fontsize=13)

plt.savefig(plot_dir+'variance_comparison_ngc.pdf',bbox_inches='tight')

# 4) Plot Data
- We plot a selection of 4PCF multiplets, $\zeta_{\ell_1\ell_2\ell_3}(r_1,r_2,r_3)$ below.
- The eventual analysis makes use of all even-parity multiplets given the above restrictions.
- The three radial bins are collapsed into one-dimension, with the restriction $r_1<r_2<r_3$.
- The first panel gives the radial bin centers and the subsequent panels give the multiplets, normalized by $r_1r_2r_3$.

In [ ]:
xmax = n_radial-1
r123 = (radii[0]*radii[1]*radii[2])[radial_filt]

# Choose which multiplet indices to plot
which_indices = [0,13,21,25]
n_plots = len(which_indices)

# Plot radial bins in first panel
fig,ax = plt.subplots(n_plots+1,figsize=(12,4*n_plots),sharex=True)
ax[0].plot(np.arange(n_radial),radii[0][radial_filt],label='Bin 1')
ax[0].plot(np.arange(n_radial),radii[1][radial_filt],label='Bin 2')
ax[0].plot(np.arange(n_radial),radii[2][radial_filt],label='Bin 3')
ax[0].set_xlim([-0.5,xmax+0.5])
ax[0].set_ylabel(r'$\bar r_i$ [$h\,\mathrm{Mpc}^{-1}$]',fontsize=13)
ax[0].legend(loc='lower right',fontsize=12)

# Define mean and variance of mocks
fourpcfN_mean = np.mean(filt_fourpcfN,axis=0)
fourpcfN_std = np.std(filt_fourpcfN,axis=0)
fourpcfS_mean = np.mean(filt_fourpcfS,axis=0)
fourpcfS_std = np.std(filt_fourpcfS,axis=0)

# Iterate over multiplets to plot
for ii,i in enumerate(which_indices):
    
    # Plot Patchy data
    ax[ii+1].fill_between(np.arange(n_radial),r123*(fourpcfN_mean[i]-fourpcfN_std[i]),
                     r123*(fourpcfN_mean[i]+fourpcfN_std[i]),color='b',alpha=0.1)
    ax[ii+1].plot(np.arange(n_radial),r123*fourpcfN_mean[i],label=r'Patchy: NGC',c='b')

    ax[ii+1].fill_between(np.arange(n_radial),r123*(fourpcfS_mean[i]-fourpcfS_std[i]),
                     r123*(fourpcfS_mean[i]+fourpcfS_std[i]),color='r',alpha=0.1)
    ax[ii+1].plot(np.arange(n_radial),r123*fourpcfS_mean[i],label=r'Patchy: SGC',c='r')
        
    # Plot BOSS data
    ax[ii+1].errorbar(np.arange(n_radial)+0.1,r123*filt_fourpcf_bossN[i],yerr=r123*fourpcfN_std[i],
                     ls='',c='b',marker='.',label=r'BOSS: NGC',alpha=0.8)
    ax[ii+1].errorbar(np.arange(n_radial)-0.1,r123*filt_fourpcf_bossS[i],yerr=r123*fourpcfS_std[i],
                     ls='',c='r',marker='x',label=r'BOSS: SGC',alpha=0.5)

    ax[ii+1].hlines(0,0,n_radial,linestyles='--',color='k',alpha=0.4)
    
    if ii==0: ax[ii+1].legend(loc='lower right',fontsize=12)
    ax[ii+1].set_ylabel(r'$r_1r_2r_3\;\zeta_{\ell_1\ell_2\ell_3}(r_1,r_2,r_3)$',fontsize=13)
    ax[ii+1].set_title(r'$(%d,%d,%d)$'%(ells[0][ang_filt][i],ells[1][ang_filt][i],ells[2][ang_filt][i]),fontsize=14);

ax[n_plots].set_xlabel(r'Radial Bin Index',fontsize=13)
fig.savefig(plot_dir+'4pcf_main_plot.pdf',bbox_inches='tight')

## 5) Analyze with Rescaled Theory Covariance

- One way in which to analyze the data is to use the *theoretical* covariance matrix to form the $\chi^2$:
$$\chi^2 = \zeta^T \mathsf{C}_{\rm theory}^{-1}\zeta$$
- We can improve the fit of theory and sample covariances by adjusting the effective survey volume and shot-noise.
- As in Hou et al. (2021), this is fit using the Kullback-Leibler divergence, comparing the true and theory covariances. Since the fitting procedure is expensive, we provide only the fitted covariances here.
- We caution that this is **not** a robust way to analyze the data. As shown in Philcox, Hou & Slepian (2021), it leads to a significant misdetection of non-Gaussianity even when none is present. 
- The results are included for completeness here.

In [ ]:
def load_rescaled_theory_cov(patch):
    cov_file = data_dir+'fitted_gaussian_cov_patchy_%s.cov'%patch
    cov_dat = pickle.load(open(cov_file, "rb"))

    # Construct covariance
    theory_cov = np.zeros((n_mult*n_radial,n_mult*n_radial))
    for i in range(n_mult):
        lam1 = '%d%d%d'%(ells[0][ang_filt][i],ells[1][ang_filt][i],ells[2][ang_filt][i])
        for j in range(n_mult):
            lam2 = '%d%d%d'%(ells[0][ang_filt][j],ells[1][ang_filt][j],ells[2][ang_filt][j])
            try: 
                this_cov = cov_dat[lam1,lam2]
            except KeyError:
                try:
                    this_cov = cov_dat[lam2,lam1].T
                except KeyError:
                    print("Covariance element %s,%s has not been computed!"%(lam1,lam2))
            theory_cov[i*n_radial:(i+1)*n_radial,j*n_radial:(j+1)*n_radial] = this_cov[radial_filt][:,radial_filt]
    return theory_cov

### Load fitted theory covariances
resc_covN = load_rescaled_theory_cov('ngc')
resc_covS = load_rescaled_theory_cov('sgc')

print("Inverting fitted theory covariances")
inv_resc_covN = np.linalg.inv(resc_covN)
inv_resc_covS = np.linalg.inv(resc_covS)

# Compute chi^2 using this rescaled model
print("Computing chi^2")
resc_chi2N = np.sum(filt_flat_fourpcfN.T*np.matmul(inv_resc_covN,filt_flat_fourpcfN.T),axis=0)
resc_chi2S = np.sum(filt_flat_fourpcfS.T*np.matmul(inv_resc_covS,filt_flat_fourpcfS.T),axis=0)

resc_chi2_bossN = np.sum((filt_flat_fourpcf_bossN)*np.matmul(inv_resc_covN,filt_flat_fourpcf_bossN))
resc_chi2_bossS = np.sum((filt_flat_fourpcf_bossS)*np.matmul(inv_resc_covS,filt_flat_fourpcf_bossS))

**Compare sample and theory covariances**
- We plot the ratio of the Patchy sample covariance to the theory covariance diagonal.
- Including the rescaling significantly improves the fit, but it is still imperfect, particularly on small scales.

In [ ]:
plt.plot(np.diag(covN)/np.diag(theory_covN),label='Theory')
plt.plot(np.diag(covN)/np.diag(resc_covN),label='Rescaled Theory')
plt.ylabel(r'$\mathrm{diag}(C_{\rm Sample})/\mathrm{diag}(C_{\rm Theory})$',fontsize=14)
plt.legend(fontsize=13,loc='upper right')
plt.xlim([0,n_max*n_radial])
plt.ylim([0,3])
for i in range(n_max):
    if i!=0: plt.vlines(n_radial*i,0,4,linestyles='--',alpha=0.4)
    plt.text(n_radial*i+n_radial/10,0.3,'%d%d%d'%(ells[0][ang_filt][i],ells[1][ang_filt][i],ells[2][ang_filt][i]),color='darkgreen',size=10) 
plt.xlabel(r'Bin Index',fontsize=14)
plt.savefig(plot_dir+'fitted_variance_comparison.pdf',bbox_inches='tight')

**Plot the detection PDFs for the NGC and SGC region separately**
- This uses the fitted-theory $\chi^2$s, and is thus not expected to be accurate.
- Both the empirical distribution of Patchy mocks and the BOSS data are plotted.
- We see the distribution widths are *very* incorrect here, due to the poor choice of covariance.

In [ ]:
bins = 25
plt.figure()

# Plot histograms
ct,x,_=plt.hist(resc_chi2N,bins=bins,alpha=0.4,density=1,label=r'%d Patchy Mocks - NGC'%n_mocks,color='r')
ct,x,_=plt.hist(resc_chi2S,bins=bins,alpha=0.4,density=1,label=r'%d Patchy Mocks - SGC'%n_mocks,color='b')

# Create PDFs of null hypothesis (i.e. chi^2)
x_arr = np.arange(min(x)*0.8,max(x)*1.4)
p = len(filt_flat_fourpcfN[0])
chi2_pdf = chi2.pdf(x_arr,p)

# Measure the BOSS detection significance
boss_probN = chi2.cdf(resc_chi2_bossN,p)
boss_probS = chi2.cdf(resc_chi2_bossS,p)

ymax = max([max(ct),max(chi2_pdf)])*1.05
print("Chi2-BOSS/N_DoF: %.2f (NGC) %.2f (SGC)"%(resc_chi2_bossN/len(filt_flat_fourpcf_bossN),resc_chi2_bossS/len(filt_flat_fourpcf_bossS)))
plt.title(r'BOSS Detection Probability: %.1f%% (NGC) %.1f%% (SGC)'%(100.*boss_probN,100.*boss_probS),fontsize=14)
print("Effective sigmas: %.2f (NGC) %.2f (SGC)"%(np.sqrt(-2.*np.log(1.-boss_probN)),np.sqrt(-2.*np.log(1.-boss_probS))))

# Plot null hypothesis and BOSS results
plt.plot(x_arr,chi2_pdf,color='orange',label=r'$\chi^2$, $N_\mathrm{DoF} = %d$'%p)
plt.vlines(resc_chi2_bossN,0,ymax,color='r',linestyles='--',label='BOSS Data - NGC')
plt.vlines(resc_chi2_bossS,0,ymax,color='b',linestyles='--',label='BOSS Data - SGC')
plt.legend(fontsize=12,bbox_to_anchor=(1.,1.));
plt.ylim([0,ymax])
plt.xlabel(r'Theoretical Covariance $\chi^2$',fontsize=14)
plt.ylabel(r'PDF',fontsize=14)
plt.savefig(plot_dir+'fitted_chi2_ngc_sgc.pdf',bbox_inches='tight')

# 6) Perform data compression

- We now analyze the data using the compression scheme in Philcox et al. (2021), based on Scoccimarro (2000).
- Data are projected onto the eigenvectors of the *model* inverse covariance matrix.
- This would be an optimal projection if true = model covariance, in the Gaussian limit, and is always unbiased.
- Here, $v = U^T\zeta$ where $\tilde{\mathsf C} = U\Lambda U^T$, and we keep the first $N_\mathrm{eig}$ eigenvectors, which are ordered in signal-to-noise.
- The covariance matrix of $v$ is *close* to diagonal, with $\langle vv^T \rangle = U^TC_DU$. If $C_D = \tilde{\mathsf{C}} = U\Lambda U^T$, then $\langle vv^T \rangle = \Lambda$, which is diagonal.

- This is performed for NGC and SGC independently, since they are assumed to be independent.

In [ ]:
# Define signal mean (used for S/N vectors)
fourpcf_meanN = np.mean(filt_flat_fourpcfN,axis=0)
fourpcf_meanS = np.mean(filt_flat_fourpcfS,axis=0)

# Compute (squared) signal-to-noise of each eigenvector
SN_N = np.matmul(evecsN.T,fourpcf_meanN)**2.*evalsN
SN_S = np.matmul(evecsS.T,fourpcf_meanS)**2.*evalsS

# Order the eigenvectors in termms of signal-to-noise
orderN = np.argsort(SN_N)[::-1]
orderS = np.argsort(SN_S)[::-1]
evalsN = evalsN[orderN]
evalsS = evalsS[orderS]
evecsN = evecsN[:,orderN]
evecsS = evecsS[:,orderS]

def project_data(data,N_eig,patch='ngc'):
    """Project data onto the eigenvector basis for NGC or SGC."""
    if patch=='ngc':
        these_evecs = evecsN[:,:N_eig]
    else:
        these_evecs = evecsS[:,:N_eig]
    proj_data = np.matmul(these_evecs.T,data.T).T
    return proj_data

# Project onto the basis vectors for Patchy and BOSS
N_eig = 50
projected_fourpcfN = project_data(filt_flat_fourpcfN,N_eig,'ngc')
projected_fourpcfS = project_data(filt_flat_fourpcfS,N_eig,'sgc')
projected_fourpcf_bossN = project_data(filt_flat_fourpcf_bossN,N_eig,'ngc')
projected_fourpcf_bossS = project_data(filt_flat_fourpcf_bossS,N_eig,'sgc')

**Compressed Covariance**
- We plot the covariance matrix of the compressed 4PCF. 
- This should be close to diagonal if the projection is optimal.

In [ ]:
plt.imshow(1e5*np.cov(projected_fourpcfN.T),cmap=cm.RdBu_r,vmax=2,vmin=-2);
fs = 14
cbar = plt.colorbar();
plt.xlabel(r'Eigenvalue Index',fontsize=fs)
plt.ylabel(r'Eigenvalue Index',fontsize=fs)
cbar.set_label(r'$10^5\times$ Projected Covariance',fontsize=fs)
plt.savefig(plot_dir+'projected_4pcf_covN.pdf',bbox_inches='tight')

**Compressed Data**
- We visualize the compressed data directly. 
- A robust detection of non-Gaussianity is seen from data-points that are well separated from zero (either above or below).

In [ ]:
meanN = projected_fourpcfN.mean(axis=0)
stdN = projected_fourpcfN.std(axis=0)
meanS = projected_fourpcfS.mean(axis=0)
stdS = projected_fourpcfS.std(axis=0)
i_arr = np.arange(len(meanN))
plt.errorbar(i_arr,meanN,label='Patchy: NGC',c='b')
plt.errorbar(i_arr,meanS,label='Patchy: SGC',c='r')
plt.errorbar(i_arr,projected_fourpcf_bossN,ls='',marker='x',alpha=0.8,
             yerr=stdN,label='BOSS Data',c='b')
plt.errorbar(i_arr,projected_fourpcf_bossS,ls='',marker='x',alpha=0.5,
             yerr=stdS,label='BOSS Data',c='r')
plt.fill_between(i_arr,meanN-stdN,meanN+stdN,alpha=0.1,color='b')
plt.fill_between(i_arr,meanS-stdS,meanS+stdS,alpha=0.1,color='r')
plt.xlabel(r'Eigenvalue Index',fontsize=14)
plt.ylabel(r'Projected 4PCF',fontsize=14)
plt.xlim([-0.5,N_eig-0.5])
plt.legend(fontsize=12)
vmax = np.max([np.max(projected_fourpcf_bossS+stdS),np.max(projected_fourpcf_bossN+stdN)])
plt.ylim([-vmax*2,vmax*2])
plt.savefig(plot_dir+'projected_4pcf_data.pdf',bbox_inches='tight')

## 7) Perform a Gaussian hypothesis test for the projected statistic

- Given projected data $v$, we form the test statistic:

$$T^2 = v^T\hat{\mathsf{C}}_v^{-1}v,$$
where $\hat{\mathsf{C}}$ is the projected sample covariance matrix, and we assume zero sample mean, matching the null hypothesis of no non-Gaussianity. 
- As in Sellentin & Heavens (2016), this follows a modified $F$-distribution, such that
$$T^2\sim \frac{\Gamma\left(\frac{n+1}{2}\right)}{\Gamma(p/2)\Gamma[(n-p+1)/2]}\frac{n^{-p/2}(T^2)^{p/2-1}}{(T^2/n+1)^{(n+1)/2}}$$
where $p = \mathrm{dim}(v)$ and $n = N_\mathrm{mocks}-1$.
- We can also form the conventional $\chi^2$ statistic, including the Hartlap factor required to debias noisy inverse covariance matrices:
$$H^2 = f_H\times v^T\hat{\mathsf{C}}^{-1}v,$$ where $f_H = (n-p-1)/n$. This is usually assumed to follow $\chi^2$ statistics, *i.e.* $H^2\sim \chi^2_p$, but this breaks down at small $n$.
- Both distributions are considered below. To form empirical distributions we apply jackknifing, computing the covariance from $(N-1)$ mocks and repeating.
- Below, analysis is performed for several choices of $N_\mathrm{eig}$, and we plot the detection PDFs and CDFs.
- The independent NGC and SGC measurements are summed, with their theoretical distributions obtained via a convolution.

In [ ]:
# List of numbers of basis vectors to test
N_eigs = [10,50]

# Number of histogram bins
bins = 25

fig1,ax1 = plt.subplots(1,len(N_eigs),figsize=(5*len(N_eigs),4))
fig2,ax2 = plt.subplots(1,len(N_eigs),figsize=(5*len(N_eigs),4),sharey=True)

for nn,N_eig in enumerate(N_eigs):
    print("\n\nN_eig = %d"%N_eig)
    
    # Project the data onto the basis
    projected_fourpcfN = project_data(filt_flat_fourpcfN,N_eig,'ngc')
    projected_fourpcfS = project_data(filt_flat_fourpcfS,N_eig,'sgc')
    projected_fourpcf_bossN = project_data(filt_flat_fourpcf_bossN,N_eig,'ngc')
    projected_fourpcf_bossS = project_data(filt_flat_fourpcf_bossS,N_eig,'sgc')
    
    ### Compute distribution of mock data using sets of (N-1) mocks, i.e. jackknifing
    this_n_mocks = len(projected_fourpcfN)-1
    
    all_T2, all_H2 = [],[]
    print("Jackknifing to compute empirical Patchy covariances. This may take a while...")
    for i in range(len(projected_fourpcfN)):
        if (i+1)%25==0: print("On step %d of %d"%(i+1,len(projected_fourpcfN)))

        # Define data
        projected_mockN = projected_fourpcfN[i]
        projected_mockS = projected_fourpcfS[i]

        # Define sample covariance in projected space
        projected_covN = np.cov(projected_fourpcfN[np.arange(len(projected_fourpcfN))!=i].T)
        projected_covS = np.cov(projected_fourpcfS[np.arange(len(projected_fourpcfS))!=i].T)
        
        # Invert the covariance
        inv_projected_covN = np.linalg.inv(projected_covN)
        inv_projected_covS = np.linalg.inv(projected_covS)

        # Compute the sample statistic
        T2_statistic = np.inner(projected_mockN,np.inner(inv_projected_covN,projected_mockN))
        T2_statistic += np.inner(projected_mockS,np.inner(inv_projected_covS,projected_mockS))
        
        all_T2.append(T2_statistic)
        
    all_T2 = np.asarray(all_T2)
    
    # Compute also Hartlap-rescaled statistic
    hartlap_factor = (this_n_mocks-N_eig-2.)/(this_n_mocks-1)
    print("Hartlap Factor for jackknifed PDF: %.3f"%hartlap_factor)
    all_H2 = all_T2*hartlap_factor
    
    ### Compute T2 and H2 statistics on BOSS data using every mmock
    projected_covN = np.cov(projected_fourpcfN.T)
    projected_covS = np.cov(projected_fourpcfS.T)
    inv_projected_covN = np.linalg.inv(projected_covN)
    inv_projected_covS = np.linalg.inv(projected_covS)
    T2_boss = np.inner(projected_fourpcf_bossN,np.inner(inv_projected_covN,projected_fourpcf_bossN))
    T2_boss += np.inner(projected_fourpcf_bossS,np.inner(inv_projected_covS,projected_fourpcf_bossS))
    hartlap_factor = (n_mocks-N_eig-2.)/(n_mocks-1)
    print("Hartlap Factor for BOSS: %.3f"%hartlap_factor)
    H2_boss = T2_boss*hartlap_factor
    
    ### Compute PDFs and CDFs (easiest to do CDFs numerically with a fine grid)
    mmax = max(all_T2)*1.5
    x_arr = np.linspace(0.001,mmax,10000)
    p = N_eig
    n = n_mocks-1
    chi2_single_pdf = chi2.pdf(x_arr,p)
    T2_single_pdf = np.float128(np.exp((p/2.-1.)*np.log(x_arr)-p/2.*np.log(n)-0.5*(n+1.)*np.log(x_arr/n+1.))*np.exp(loggamma((n+1.)/2.)-loggamma(p/2.)-loggamma((n-p+1.)/2.)))

    # Compute PDF of H^2_NGC + H^2_SGC using a convolution
    chi2_pdf = np.fft.ifft(np.fft.fft(chi2_single_pdf)**2.).real
    chi2_pdf /= np.sum(chi2_pdf)*np.diff(x_arr)[0]

    # Compute PDF of T^2_NGC + T^2_SGC using a convolution
    T2_pdf = np.fft.ifft(np.fft.fft(T2_single_pdf)**2.).real
    T2_pdf /= np.sum(T2_pdf)*np.diff(x_arr)[0]

    # Compute CDFs
    chi2_cdf = interp1d(x_arr,np.cumsum(chi2_pdf)*np.diff(x_arr)[0])
    T2_cdf = interp1d(x_arr,np.cumsum(T2_pdf)*np.diff(x_arr)[0])

    ### Plot empirical distributions
    ct,_,_=ax1[nn].hist(all_H2,bins=bins,alpha=0.4,density=1,color='orange')#,label=r'$H^2$')
    ax1[nn].hist(all_T2,bins=bins,alpha=0.4,density=1,color='blue')#,label=r'$T^2$');
    ax1[nn].vlines(T2_boss,0,2*mmax,color='blue',linestyles='--')
    ax1[nn].vlines(H2_boss,0,2*mmax,color='orange',linestyles='--')
    
    ### Plot theoretical distributions
    ax1[nn].plot(x_arr,chi2_pdf,color='orange',label=r'$H^2$')
    ax1[nn].plot(x_arr,T2_pdf,color='blue',label=r'$T^2$')
    
    ### Add cosmetics
    if nn==0:
        ax1[nn].legend(fontsize=12);
    ax1[nn].set_xlim([0,mmax])
    ax1[nn].set_ylim([0,max([max(ct),max(chi2_pdf)])*1.05])
    ax1[nn].set_xlabel(r'$H^2$ or $T^2$',fontsize=14)
    if nn==0: ax1[nn].set_ylabel(r'PDF',fontsize=14)
    ax1[nn].set_title(r'$N_\mathrm{eig} = %d$'%N_eig,fontsize=14);
    if nn==2: fig1.savefig(plot_dir+'projected_pdfs_all.pdf',bbox_inches='tight')

    ### Compute detection probabilities, as a 1-tail test
    prob_T2 = T2_cdf(all_T2)
    prob_chi2 = chi2_cdf(all_H2)

    prob_T2_boss = T2_cdf(T2_boss)
    prob_chi2_boss = chi2_cdf(H2_boss)

    # Count fraction of false detections 
    N_detections_T2 = (np.sum(prob_T2>0.95))*1./n_mocks
    N_detections_chi2 = (np.sum(prob_chi2>0.95))*1./n_mocks

    print("\nFraction of 2-sigma detections with %d eigenvalues"%N_eig)
    print("chi^2 + Hartlap model: %.3f"%N_detections_chi2)
    print("T^2: %.3f"%N_detections_T2)
    print("Expected: %.3f"%(0.05))

    print("\nBOSS Detection Rates:")
    print("chi^2 + Hartlap model: %.3f"%prob_chi2_boss)
    print("T^2: %.3f"%prob_T2_boss)
    print("Effective Sigmas: %.2f"%np.sqrt(-2.*np.log(1.-prob_T2_boss)))

    ### Histogram the detection PDFs
    ax2[nn].set_title(r'BOSS: %.1f%%'%(100.*prob_T2_boss),fontsize=14)
    ct,_,_=ax2[nn].hist(prob_chi2,bins=bins,histtype='step',color='orange',label=r'$H^2$',density=0,range=[0,1.00001]);
    ax2[nn].hist(prob_T2,bins=bins,histtype='step',color='blue',label=r'$T^2$',density=0,range=[0,1.00001]);
    ax2[nn].hlines(n_mocks/bins,0,1,linestyles='--')
    ax2[nn].vlines(prob_chi2_boss,0,max(ct)*1.2,color='orange',linestyles='--')
    ax2[nn].vlines(prob_T2_boss,0,max(ct)*1.2,color='blue',linestyles='--')
    ax2[nn].set_xlim([0.0,1])
    ax2[nn].fill_betweenx(np.arange(0,max(ct)*2),0.95,1,alpha=0.1,color='k')
    ax2[nn].set_ylim([0,max(ct)*1.1])
    if nn==0: ax2[nn].legend(fontsize=12,loc='lower right');
    if nn==0: ax2[nn].set_ylabel(r'Occurences',fontsize=14)
    ax2[nn].set_xlabel(r'Detection Probability',fontsize=14);
    if nn==2:
        fig2.savefig(plot_dir+'projected_cdfs_all.pdf',bbox_inches='tight')

### This completes the analysis